In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
from time import time

In [5]:
tick = time()

In [7]:
mnist  = input_data.read_data_sets("/tmp/data",one_hot =True)

In [8]:
type(mnist)

In [9]:
mnist.train.images.shape

In [10]:
sample  = mnist.train.images[2].reshape(28,28)

In [11]:
plt.imshow(sample,cmap ='Greys')

In [13]:
batch_size = 100
n_classes = 10
n_samples = mnist.train.num_examples
n_input  = 784
n_output =10

In [14]:
def multilayer_perceptron(x,weights,biases):
    
    '''
    x:Placeholder for data input
    weights : dictionary for weights
    biases : dict of bias values
    '''
    #First hidden Layer activation RELU activation 
    #x*w + bias
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    # RELU(x*w+bias) -> f(x) = max(0,x)
    layer_1 = tf.nn.relu(layer_1) 
    
    
    #Second hidden layer
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2) 
    
    #Output layer
    out_layer = tf.matmul(layer_2,weights['out']) + biases['out']
    
    return out_layer

In [15]:
sess = tf.InteractiveSession()

In [17]:
def model(training_epochs,l,n_hidden_1,n_hidden_2):
        
        weights= {
    'h1': tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2,n_classes]))
        }
        
        biases= {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
        }
        x = tf.placeholder('float',[None,n_input])
        y = tf.placeholder('float',[None, n_classes])
        pred = multilayer_perceptron(x,weights,biases)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( logits=pred, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=l).minimize(cost)
        #initializing variables
        init = tf.initialize_all_variables()
        sess.run(init)
        learning_rate = l
        for epoch in range(training_epochs):
                #cost
            avg_cost= 0.0

            total_batch = int(n_samples/batch_size) #55000/100 batches total

            for i in range(total_batch):

                batch_x,batch_y = mnist.train.next_batch(batch_size)

                _,c = sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})

                avg_cost += c/total_batch

            print("Epoch : {}  cost{:4f}".format(epoch+1,avg_cost ) ) 

        print("Model has completed {} epochs of training".format(training_epochs))
        correct_predictions = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
        correct_predictions = tf.cast(correct_predictions,'float')
        accuracy = tf.reduce_mean(correct_predictions)
        acc = accuracy.eval({x:mnist.test.images,y:mnist.test.labels})
        error = 1 - acc
        print("**********************************")
        print("Accuracy : {}".format(acc))    
        print("Error : {}".format(error))
        print("**********************************")
        return error

In [18]:
t = mnist.train.next_batch(1)

In [19]:
Xsamp , ysamp = t

In [20]:
plt.imshow(Xsamp.reshape(28,28),cmap='Greys')

In [21]:
ysamp

In [23]:
import numpy as np
import pandas as pd
df = pd.DataFrame(columns=['learning_rate','hidden_layer_neurons','error'])
learning_rates = np.array([0.001,0.005,0.01,0.05,0.1])
hidden_count = np.array([64, 128, 256, 512, 1024])
for i in learning_rates:
    print("Learning rate : {} ".format(i))
    print("----------------------------------------------------------")
    print("----------------------------------------------------------")    
    for j in hidden_count:
        print("Number of neurons in hidden layers : {} ".format(j))
        print("_____________________________________________________________")
        error = model(5,i,j,j)
        df = df.append({'learning_rate': i,'hidden_layer_neurons':j,'error':error}, ignore_index=True)

In [24]:
df

In [25]:
df[df.error<0.2].pivot(index='learning_rate',columns='hidden_layer_neurons',values='error').plot()
display()

In [26]:
mnist.test.labels

In [27]:
mnist.test.images

In [28]:
tock = time()
print(tock - tick)